# Lesson 4: Form Parsing

**Lesson objective**: Incorporate form parsing to the workflow

In your previous lesson, you used LlamaParse to parse a resume, and included parsing instructions. You'll do that again this time, but the instructions are going to be more advanced -- you're going to get it to read an application form and convert it into a list of fields that need to be filled in, and return that as a JSON object. You will then incorporate these steps in the workflow you started building in the previous lesson.

In [21]:
import os, json
from llama_index.llms.gemini import Gemini
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.core import (
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage
)

from llama_index.core.workflow import (
    StartEvent,
    StopEvent,
    Workflow,
    step,
    Event,
    Context
)
from IPython.display import display, HTML
from llama_index.utils.workflow import draw_all_possible_flows

In [12]:
import nest_asyncio
nest_asyncio.apply()

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>To access <code>fake_application_form.pdf</code>, <code>fake_resume.pdf</code>, <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. The form and resume are inside the data folder.

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips and Help"</em> Lesson.</p>

</div>

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI chat models can vary with each execution due to their dynamic, probabilistic nature. Don't be surprised if your results differ from those shown in the video.</p>

## Parsing an Application Form with LlamaParse

Let's create a parser with our new parsing instructions, including formatting instructions.

In [ ]:
import fitz  # PyMuPDF
from llama_index.core.schema import Document
from llama_index.core.node_parser import SentenceSplitter
# Step 1: Parse the PDF
def parse_pdf(file_path: str) -> str:
    doc = fitz.open(file_path)
    text = ''
    for page in doc:
        text += page.get_text("text") + '\n'
    return text
# Step 2: Create Document from parsed text
def create_document_from_text(text: str) -> Document:
    return Document(text=text)
# Step 3: Split Document into Nodes
def split_document_into_nodes(document: Document) -> list:
    splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=20)
    nodes = splitter.get_nodes_from_documents([document])
    return nodes
# Example usage
cv_parser = parse_pdf('data/fake_resume.pdf')
document = create_document_from_text(cv_parser)
nodes_cv = split_document_into_nodes(document)



cv_application = parse_pdf('data/fake_application_form.pdf')
document = create_document_from_text(cv_application)
nodes_app = split_document_into_nodes(document)



In [ ]:
nodes_app[0].text, nodes_cv[0].text

('Big Tech Co. Job Application Form \nPosition: Senior Web Developer C3 \nThanks for applying to Big Tech Co.! We are humbled that you would consider working here. \nPlease fill in the following form to help us get started. \n \nFirst Name \n \nLast Name  \n \nEmail \n \nPhone \n \nLinkedin \n \n \nProject Portfolio \n \n \nDegree \n \nGraduation \nDate \n \nCurrent Job title \n \nCurrent \nEmployer \n \nTechnical Skills \n \n \nDescribe why \nyou’re a good fit \nfor this position \n \nDo you have 5 \nyears of \nexperience in \nReact?',
 "\u202dSarah Chen\u202c\n\u202dFull Stack Web Developer\u202c\n\u202dEmail:\u202c\u202dsarah.chen@email.com\u202c\n\u202dLinkedIn:\u202c\u202dlinkedin.com/in/sarahchen\u202c\n\u202dGitHub:\u202c\u202dgithub.com/sarahcodes\u202c\n\u202dPortfolio: sarahchen.dev\u202c\n\u202dLocation: San Francisco, CA\u202c\n\u202dProfessional Summary\u202c\n\u202dInnovative Full Stack Web Developer with 6+ years of experience crafting scalable web\u202c\n\u202dapplicati

A useful thing LLMs can do is turn human-readable formats into machine-readable ones. You will ask the LLM to turn the list into a JSON object with a list of fields.

In [27]:
from dotenv import load_dotenv
load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

from llama_index.llms.google_genai import GoogleGenAI

llm = GoogleGenAI(model="models/gemini-2.0-flash")

In [29]:
raw_json = llm.complete(
    f"""
    This is a parsed form.
    Convert it into a JSON object containing only the list 
    of fields to be filled in, in the form {{ fields: [...] }}. 
    <form>{nodes_app}</form>. 
    Return JSON ONLY, no markdown."""
)

Here's the raw JSON:

In [30]:
raw_json.text

'```json\n{\n  "fields": [\n    "First Name",\n    "Last Name",\n    "Email",\n    "Phone",\n    "Linkedin",\n    "Project Portfolio",\n    "Degree",\n    "Graduation Date",\n    "Current Job title",\n    "Current Employer",\n    "Technical Skills",\n    "Describe why you’re a good fit for this position",\n    "Do you have 5 years of experience in React?"\n  ]\n}\n```'

And you can print out the fields one by one.

In [ ]:
fields = json.loads(raw_json.text)["fields"]

for field in fields:
    print(field)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

Now that you know how to parse the job application form, you will add this processing to the workflow of lesson 3. You will do that in two steps as shown in this figure:

<img width="700" src="images/L4-diagrams.png">

## Adding a Form Parser to the Workflow (first update)

Now let's take the workflow you built in the last lesson and add your parser.

In [ ]:
class ParseFormEvent(Event):
    application_form: str

class QueryEvent(Event):
    query: str

Your `set_up` step now emits a `ParseFormEvent` which triggers your new step, `parse_form`. For the moment you can leave the ask_questions step untouched, it will be updated in another section of this notebook.

In [ ]:
class RAGWorkflow(Workflow):
    
    storage_dir = "./storage"
    llm: OpenAI
    query_engine: VectorStoreIndex

    @step
    async def set_up(self, ctx: Context, ev: StartEvent) -> ParseFormEvent:

        if not ev.resume_file:
            raise ValueError("No resume file provided")

        if not ev.application_form:
            raise ValueError("No application form provided")

        # define the LLM to work with
        self.llm = OpenAI(model="gpt-4o-mini")

        # ingest the data and set up the query engine
        if os.path.exists(self.storage_dir):
            # you've already ingested the resume document
            storage_context = StorageContext.from_defaults(persist_dir=self.storage_dir)
            index = load_index_from_storage(storage_context)
        else:
            # parse and load the resume document
            documents = LlamaParse(
                api_key=llama_cloud_api_key,
                base_url=os.getenv("LLAMA_CLOUD_BASE_URL"),
                result_type="markdown",
                content_guideline_instruction="This is a resume, gather related facts together and format it as bullet points with headers"
            ).load_data(ev.resume_file)
            # embed and index the documents
            index = VectorStoreIndex.from_documents(
                documents,
                embed_model=OpenAIEmbedding(model_name="text-embedding-3-small")
            )
            index.storage_context.persist(persist_dir=self.storage_dir)

        # create a query engine
        self.query_engine = index.as_query_engine(llm=self.llm, similarity_top_k=5)

        # you no longer need a query to be passed in, 
        # you'll be generating the queries instead 
        # let's pass the application form to a new step to parse it
        return ParseFormEvent(application_form=ev.application_form)

    @step
    async def parse_form(self, ctx: Context, ev: ParseFormEvent) -> QueryEvent:
        parser = LlamaParse(
            api_key=llama_cloud_api_key,
            base_url=os.getenv("LLAMA_CLOUD_BASE_URL"),
            result_type="markdown",
            content_guideline_instruction="This is a job application form. Create a list of all the fields that need to be filled in.",
            formatting_instruction="Return a bulleted list of the fields ONLY."
        )

        # get the LLM to convert the parsed form into JSON
        result = parser.load_data(ev.application_form)[0]
        raw_json = self.llm.complete(
            f"""
            This is a parsed form. 
            Convert it into a JSON object containing only the list 
            of fields to be filled in, in the form {{ fields: [...] }}. 
            <form>{result.text}</form>. 
            Return JSON ONLY, no markdown.
            """)
        fields = json.loads(raw_json.text)["fields"]

        for field in fields:
            print(field)
        return StopEvent(result="Dummy event")

    # will be edited in the next section
    @step
    async def ask_question(self, ctx: Context, ev: QueryEvent) -> StopEvent:
        response = self.query_engine.query(f"This is a question about the specific resume we have in our database: {ev.query}")
        return StopEvent(result=response.response)

In [ ]:
w = RAGWorkflow(timeout=60, verbose=False)
result = await w.run(
    resume_file="data/fake_resume.pdf",
    application_form="data/fake_application_form.pdf"
)

## Generating Questions (second update)

Excellent! Your workflow knows what fields it needs answers for. In this next iteration, you can fire off one `QueryEvent` for each of the fields, so they'll be executed concurrently (we talked about doing concurrent steps in Lesson 2). 

<img width="700" src="images/L4-diag-2.png">


The changes you're going to make are:
* Generate a `QueryEvent` for each of the questions you pulled out of the form
* Create a `fill_in_application` step which will take all the responses to the questions and aggregate them into a coherent response
* Add a `ResponseEvent` to pass the results of queries to `fill_in_application`

In [ ]:
class ParseFormEvent(Event):
    application_form: str

class QueryEvent(Event):
    query: str
    field: str

# new!
class ResponseEvent(Event):
    response: str

In [ ]:
class RAGWorkflow(Workflow):
    
    storage_dir = "./storage"
    llm: OpenAI
    query_engine: VectorStoreIndex

    @step
    async def set_up(self, ctx: Context, ev: StartEvent) -> ParseFormEvent:

        if not ev.resume_file:
            raise ValueError("No resume file provided")

        if not ev.application_form:
            raise ValueError("No application form provided")

        # define the LLM to work with
        self.llm = OpenAI(model="gpt-4o-mini")

        # ingest the data and set up the query engine
        if os.path.exists(self.storage_dir):
            # you've already ingested the resume document
            storage_context = StorageContext.from_defaults(persist_dir=self.storage_dir)
            index = load_index_from_storage(storage_context)
        else:
            # parse and load the resume document
            documents = LlamaParse(
                api_key=llama_cloud_api_key,
                base_url=os.getenv("LLAMA_CLOUD_BASE_URL"),
                result_type="markdown",
                content_guideline_instruction="This is a resume, gather related facts together and format it as bullet points with headers"
            ).load_data(ev.resume_file)
            # embed and index the documents
            index = VectorStoreIndex.from_documents(
                documents,
                embed_model=OpenAIEmbedding(model_name="text-embedding-3-small")
            )
            index.storage_context.persist(persist_dir=self.storage_dir)

        # create a query engine
        self.query_engine = index.as_query_engine(llm=self.llm, similarity_top_k=5)

        # you no longer need a query to be passed in, 
        # you'll be generating the queries instead 
        # let's pass the application form to a new step to parse it
        return ParseFormEvent(application_form=ev.application_form)

    @step
    async def parse_form(self, ctx: Context, ev: ParseFormEvent) -> QueryEvent:
        parser = LlamaParse(
            api_key=llama_cloud_api_key,
            base_url=os.getenv("LLAMA_CLOUD_BASE_URL"),
            result_type="markdown",
            content_guideline_instruction="This is a job application form. Create a list of all the fields that need to be filled in.",
            formatting_instruction="Return a bulleted list of the fields ONLY."
        )

        # get the LLM to convert the parsed form into JSON
        result = parser.load_data(ev.application_form)[0]
        raw_json = self.llm.complete(
            f"""
            This is a parsed form. 
            Convert it into a JSON object containing only the list 
            of fields to be filled in, in the form {{ fields: [...] }}. 
            <form>{result.text}</form>. 
            Return JSON ONLY, no markdown.
            """)
        fields = json.loads(raw_json.text)["fields"]

        # new!
        # generate one query for each of the fields, and fire them off
        for field in fields:
            ctx.send_event(QueryEvent(
                field=field,
                query=f"How would you answer this question about the candidate? {field}"
            ))

        # store the number of fields so we know how many to wait for later
        await ctx.set("total_fields", len(fields))
        return

    @step
    async def ask_question(self, ctx: Context, ev: QueryEvent) -> ResponseEvent:
        response = self.query_engine.query(f"This is a question about the specific resume we have in our database: {ev.query}")
        return ResponseEvent(field=ev.field, response=response.response)

    # new!
    @step
    async def fill_in_application(self, ctx: Context, ev: ResponseEvent) -> StopEvent:
        # get the total number of fields to wait for
        total_fields = await ctx.get("total_fields")

        responses = ctx.collect_events(ev, [ResponseEvent] * total_fields)
        if responses is None:
            return None # do nothing if there's nothing to do yet

        # we've got all the responses!
        responseList = "\n".join("Field: " + r.field + "\n" + "Response: " + r.response for r in responses)

        result = self.llm.complete(f"""
            You are given a list of fields in an application form and responses to
            questions about those fields from a resume. Combine the two into a list of
            fields and succinct, factual answers to fill in those fields.

            <responses>
            {responseList}
            </responses>
        """)
        return StopEvent(result=result)

In [ ]:
w = RAGWorkflow(timeout=120, verbose=False)
result = await w.run(
    resume_file="data/fake_resume.pdf",
    application_form="data/fake_application_form.pdf"
)
print(result)

## Workflow Visualization

You can visualize the workflow you just created.

In [ ]:
WORKFLOW_FILE = "workflows/form_parsing_workflow.html"
draw_all_possible_flows(w, filename=WORKFLOW_FILE)
html_content = extract_html_content(WORKFLOW_FILE)
display(HTML(html_content), metadata=dict(isolated=True))

## Cool!

Your workflow takes all the fields in the form and generates plausible answers for all of them. There are a couple of fields where I think it can do better, and in the next lesson you'll add the ability to give that feedback to the agent and get it to try again.